# `utils.check_var`

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import slickml

print(f"Loaded SlickML Version = {slickml.__version__}")

Loaded SlickML Version = 0.2.0b0


In [4]:
from slickml.utils import check_var

help(check_var)

Help on function check_var in module slickml.utils._base:

check_var(var: Any, *, var_name: str, dtypes: Union[type, Tuple[type]], values: Union[Any, Tuple[Any], NoneType] = None) -> None
    Validates the variable's dtype and possible value.
    
    Parameters
    ----------
    var : Any
        Variable
    
    var_name : str
        Variable name
    
    dtypes : Union[type, Tuple[type]]
        Data type classes
    
    values : Union[Any, Tuple[Any]], optional
        Possible values, by default None
    
    Returns
    -------
    None
    
    Raises
    ------
    TypeError
        If dtypes are invalid
    
    ValueError
        If values are invalid
    
    Notes
    -----
    This is the main function that is being used across the API as the variable checker before any
    class/function being instantiated. This is our solution instead of using `pydantic` validator
    and root_validator due to a lot of issues (i.e. data type casting/truncation in a silence mode)
   

### Example 1: `check_var` in validation of `class` variables

In [11]:
from dataclasses import dataclass
from slickml.utils import check_var


@dataclass
class Foo:
    var_str: str
    var_float: float = 42.0
    var_int: int = 1367

    def __post_init__(self):
        check_var(
            self.var_str,
            var_name="var_str",
            dtypes=str,
        )
        check_var(
            self.var_float,
            var_name="var_float",
            dtypes=float,
            values=(41, 42),
        )
        check_var(
            self.var_int,
            var_name="var_int",
            dtypes=int,
            values=(1367, 1400),
        )

In [12]:
f = Foo(var_str="qux")

### `class Foo` cannot be instantiated with `invalid` inputs; To fully understand the functionality of `check_var` please take a look at the `errors` below for each scenario

In [13]:
f = Foo(
    var_str=1,
)

TypeError: The input var_str must have str dtype.

In [14]:
f = Foo(
    var_str="baz",
    var_float=43.0,
)

ValueError: The input var_float must have one of (41, 42) values.

In [15]:
f = Foo(
    var_str="baz",
    var_float=42.0,
    var_int=1.0,
)

TypeError: The input var_int must have int dtype.

In [16]:
f = Foo(
    var_str="baz",
    var_float=42.0,
    var_int=1,
)

ValueError: The input var_int must have one of (1367, 1400) values.